In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pprint import pprint

from terminusdb_client import WOQLQuery as WQ

from ads_query_eval.config import get_terminus_client, get_terminus_config
from ads_query_eval.app.bootstrap import QUERIES

client = get_terminus_client()

Checking for 200 response from http://terminus:6363/api/info...


# scratch

In [13]:
from urllib.parse import quote_plus, quote

quote("2022-08-19_full:\"coronal mass ejection\"")

'2022-08-19_full%3A%22coronal%20mass%20ejection%22'

In [38]:
retrieval = "Retrieval/2022-08-19_full%3A%22coronal%20mass%20ejection%22"
q = WQ().woql_and(
    WQ().triple("v:ril", "retrieval", retrieval),
    WQ().triple("v:ri", "retrieved_items_list", "v:ril"),
    WQ().read_document("v:ri", "v:ri_doc"),
)

results = client.query(q)
results

{'@type': 'api:WoqlResponse',
 'api:status': 'api:success',
 'api:variable_names': ['ril', 'ri', 'ri_doc'],
 'bindings': [{'ri': 'RetrievedItem/008753fdc9d3582361416d4885c132dc25e556ba0bd3511c416898ebb25a2c00',
   'ri_doc': {'@id': 'RetrievedItem/008753fdc9d3582361416d4885c132dc25e556ba0bd3511c416898ebb25a2c00',
    '@type': 'RetrievedItem',
    'content': {'[citations]': {'num_citations': 39, 'num_references': 90},
     'abstract': 'Context. Coronal mass ejections (CMEs) are the primary source of strong space weather disturbances at Earth. Their geo-effectiveness is largely determined by their dynamic pressure and internal magnetic fields, for which reliable predictions at Earth are not possible with traditional cone CME models. <BR /> Aims: We study two well-observed Earth-directed CMEs using the EUropean Heliospheric FORecasting Information Asset (EUHFORIA) model, testing for the first time the predictive capabilities of a linear force-free spheromak CME model initialised using para

In [15]:
q = WQ().triple("v:s", "type", "@schema:Query")

results = client.query(q)
results
for s in [b['s'] for b in results['bindings']]:
    pprint(client.get_document(s, unfold="true"))

{'@id': 'Query/full%3A%22coronal%20mass%20ejection%22',
 '@type': 'Query',
 'query_literal': 'full:"coronal mass ejection"'}
{'@id': 'Query/full%3A%22geomagnetically%20induced%20current%22',
 '@type': 'Query',
 'query_literal': 'full:"geomagnetically induced current"'}
{'@id': 'Query/full%3A%22ionospheric_conductivity%22',
 '@type': 'Query',
 'query_literal': 'full:"ionospheric_conductivity"'}
{'@id': 'Query/full%3A%22particle%20acceleration%22',
 '@type': 'Query',
 'query_literal': 'full:"particle acceleration"'}
{'@id': 'Query/full%3A%22solar%20wind%22',
 '@type': 'Query',
 'query_literal': 'full:"solar wind"'}
{'@id': 'Query/full%3A%22space%20weather%22',
 '@type': 'Query',
 'query_literal': 'full:"space weather"'}
{'@id': 'Query/full%3A%22substorm%22',
 '@type': 'Query',
 'query_literal': 'full:"substorm"'}
{'@id': 'Query/full%3A(%22interplanetary%20magnetic%20field%22%20AND%20reconnection)',
 '@type': 'Query',
 'query_literal': 'full:("interplanetary magnetic field" AND reconnecti

In [9]:
q = WQ().triple("v:s", "type", "@schema:RetrievedItemsList")

results = client.query(q)
results
for s in [b['s'] for b in results['bindings']]:
    pprint(client.get_document(s, unfold="true"))

{'@id': 'RetrievedItemsList/d6b3b30d3844b357621d4828b33fd49a5881f6b2fc858d6dd1a4404f742bda79',
 '@type': 'RetrievedItemsList',
 'retrieval': 'Retrieval/2022-08-19_full%3A%22coronal%20mass%20ejection%22'}


# util

In [7]:
for name, defn in client.get_existing_classes().items():
    print(name)
    #pprint(defn)
    #print()

Evaluator
Uncertainty
User
RetrievableItem
Query
CredentialsRequest
EmailedCredentialsLink
Evaluation
EvaluationStatus
RetrievedItem
InviteLink
JobCategory
Operation
Retrieval
ItemOfEvaluation
EvaluatingProcedure
JobRun
OperationStatus
Relevance
RetrievedItemsList


In [ ]:
import requests

def tget(url):
    client._check_connection()
    return requests.get(
        url,
        headers=client._default_headers,
        auth=client._auth(),
    )

def tget_ttl(dbid, which=None):
    assert which in {"schema","instance"}
    rv = tget(f"http://terminus:6363/api/triples/admin/{dbid}/local/branch/main/{which}?format=turtle")
    for line in rv.json().split('\n'):
        print(line)
    return rv

#tget_ttl(client.db, which="schema")
#tget_ttl(client.db, which="instance")